In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/waste/

/content/drive/MyDrive/waste


In [3]:
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!pip install -r requirements.txt 

Cloning into 'yolov5'...
remote: Enumerating objects: 15305, done.
remote: Total 15305 (delta 0), reused 0 (delta 0), pack-reused 15305
Receiving objects: 100% (15305/15305), 14.19 MiB | 19.87 MiB/s, done.
Resolving deltas: 100% (10497/10497), done.
/content/drive/MyDrive/waste/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.1 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.3
    Uninstalling setuptools-63.4.3:
      Successfully uninstalled setuptools-63.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=

In [4]:
import os
import shutil
import numpy as np
import tqdm

In [5]:
from pycocotools.coco import COCO
data_source = COCO(annotation_file='/content/drive/MyDrive/waste/dataset/annotations.json')

loading annotations into memory...
Done (t=0.77s)
creating index...
index created!


In [6]:
catIds = data_source.getCatIds()
cats = data_source.loadCats(catIds)

In [7]:
classes = {}
taco_labels = {}
taco_labels_inverse = {}

for c in cats:
    taco_labels[len(classes)] = c['id']
    taco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

In [8]:
!mkdir -p tmp/labels tmp/images
IMAGES_PATH = 'tmp/images/'
LABELS_PATH = 'tmp/labels/'

In [9]:
imgIds = data_source.getImgIds()
# print(data_source.loadImgs(0)[0])
for index, img_id in tqdm.tqdm(enumerate(imgIds)):
    img_info = data_source.loadImgs(img_id)[0]
      # img_dir: batch_x/.....jpg ---> batch_x_......jpg
    img_dir = img_info['file_name'].replace('/', '-')
    
    image_name = img_dir.split('.')[0]
    label_dir = LABELS_PATH + image_name + '.txt'
    
    height = img_info['height']
    width = img_info['width']
    
#     print ("Copying from /kaggle/input/tacotrashdataset/data/{} to {}".format(img_info['file_name'], os.path.join(IMAGES_PATH, img_dir)))
    # get images
    shutil.copy(f"/content/drive/MyDrive/waste/dataset/{img_info['file_name']}", os.path.join(IMAGES_PATH, img_dir))
        

    
      # get labels
    with open(label_dir, mode='w') as fp:
#         print (f"Creating label_dir {label_dir} for {image_name}")
        annotation_id = data_source.getAnnIds(img_id)
        if len(annotation_id) == 0:
            fp.write('')
            continue
        boxes = np.zeros((0, 5))
        annotations = data_source.loadAnns(annotation_id)
        lines = ''
        for annotation in annotations:
            label = taco_labels_inverse[annotation['category_id']]
            box = annotation['bbox']
            # some annotations have basically no width / height (extremely small), skip them
            if box[2] < 1 or box[3] < 1:
                continue
            # top_x,top_y,width,height ----> cen_x,cen_y,width,height
            # standardize to 0-1
            box[0] = round((box[0] + box[2] / 2) / width, 6)
            box[1] = round((box[1] + box[3] / 2) / height, 6)
            box[2] = round(box[2] / width, 6)
            box[3] = round(box[3] / height, 6)
            # line: label x_center y_center width height
            lines = lines + str(label)
            for i in box:
                lines += ' ' + str(i)
            lines += '\n'
        fp.writelines(lines)

1500it [05:51,  4.26it/s]


In [10]:
# test
annotation_id = data_source.getAnnIds(1)
data_source.loadAnns(annotation_id)

[{'id': 2,
  'image_id': 1,
  'category_id': 18,
  'segmentation': [[928.0,
    1876.0,
    938.0,
    1856.0,
    968.0,
    1826.0,
    990.0,
    1808.0,
    998.0,
    1790.0,
    1069.0,
    1727.0,
    1096.0,
    1702.0,
    1159.0,
    1644.0,
    1212.0,
    1588.0,
    1258.0,
    1540.0,
    1314.0,
    1482.0,
    1357.0,
    1444.0,
    1392.0,
    1416.0,
    1409.0,
    1393.0,
    1430.0,
    1369.0,
    1415.0,
    1347.0,
    1130.0,
    1087.0,
    780.0,
    763.0,
    528.0,
    533.0,
    479.0,
    486.0,
    466.0,
    466.0,
    448.0,
    457.0,
    427.0,
    468.0,
    387.0,
    502.0,
    321.0,
    554.0,
    244.0,
    608.0,
    118.0,
    693.0,
    37.0,
    750.0,
    3.0,
    780.0,
    1.0,
    995.0,
    28.0,
    1032.0,
    104.0,
    1119.0,
    403.0,
    1471.0,
    666.0,
    1805.0,
    763.0,
    1954.0,
    782.0,
    1945.0,
    796.0,
    1970.0,
    803.0,
    1976.0,
    818.0,
    1976.0,
    836.0,
    1956.0,
    852.0,
    1954.0,

In [11]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import splitfolders
splitfolders.ratio('tmp', output='taco', seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 3000 files [00:35, 84.08 files/s]


In [13]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    print(line)
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [14]:
%%writetemplate data/taco28.yaml

train: taco/train/images
val:   taco/val/images

# number of classes
nc: 60

# class names
names: ['Aluminium foil',
'Battery',
'Aluminium blister pack',
'Carded blister pack',
'Other plastic bottle',
'Clear plastic bottle',
'Glass bottle',
'Plastic bottle cap',
'Metal bottle cap',
'Broken glass',
'Food Can',
'Aerosol',
'Drink can',
'Toilet tube',
'Other carton',
'Egg carton',
'Drink carton',
'Corrugated carton',
'Meal carton',
'Pizza box',
'Paper cup',
'Disposable plastic cup',
'Foam cup',
'Glass cup',
'Other plastic cup',
'Food waste',
'Glass jar',
'Plastic lid',
'Metal lid',
'Other plastic',
'Magazine paper',
'Tissues',
'Wrapping paper',
'Normal paper',
'Paper bag',
'Plastified paper bag',
'Plastic film',
'Six pack rings',
'Garbage bag',
'Other plastic wrapper',
'Single-use carrier bag',
'Polypropylene bag',
'Crisp packet',
'Spread tub',
'Tupperware',
'Disposable food container',
'Foam food container',
'Other plastic container',
'Plastic glooves',
'Plastic utensils',
'Pop tab',
'Rope & strings',
'Scrap metal',
'Shoe',
'Squeezable tube',
'Plastic straw',
'Paper straw',
'Styrofoam piece',
'Unlabeled litter',
'Cigarette']

data/taco28.yaml


In [15]:
%cat data/taco28.yaml


train: taco/train/images
val:   taco/val/images

# number of classes
nc: 60

# class names
names: ['Aluminium foil',
'Battery',
'Aluminium blister pack',
'Carded blister pack',
'Other plastic bottle',
'Clear plastic bottle',
'Glass bottle',
'Plastic bottle cap',
'Metal bottle cap',
'Broken glass',
'Food Can',
'Aerosol',
'Drink can',
'Toilet tube',
'Other carton',
'Egg carton',
'Drink carton',
'Corrugated carton',
'Meal carton',
'Pizza box',
'Paper cup',
'Disposable plastic cup',
'Foam cup',
'Glass cup',
'Other plastic cup',
'Food waste',
'Glass jar',
'Plastic lid',
'Metal lid',
'Other plastic',
'Magazine paper',
'Tissues',
'Wrapping paper',
'Normal paper',
'Paper bag',
'Plastified paper bag',
'Plastic film',
'Six pack rings',
'Garbage bag',
'Other plastic wrapper',
'Single-use carrier bag',
'Polypropylene bag',
'Crisp packet',
'Spread tub',
'Tupperware',
'Disposable food container',
'Foam food container',
'Other plastic container',
'Plastic glooves',
'Plastic utensils',
'Pop tab',
'Ro

In [16]:
# Removing the tmp folder

!rm -r tmp

In [17]:
import time
start = time.time()
!python train.py --img 640 --batch 16  --epochs 100 --workers 8 \
--data /content/drive/MyDrive/waste/yolov5/data/taco28.yaml \
--weights yolov5s.pt

end = time.time() 

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/waste/yolov5/data/taco28.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t